In [1]:
from datasets import Audio, load_dataset, interleave_datasets

In [2]:
dataset_names = ["mozilla-foundation/common_voice_11_0", "facebook/voxpopuli"]
configs = ["es", "es"]
splits = ["train", "train"]
text_column_names = ["sentence", "text"]
audio_column_names = ["audio", "audio"]

In [3]:
def load_multiple_streaming_datasets(dataset_names, dataset_config_names, splits=None, text_column_names=None, audio_column_names=None, **kwargs):
    if len(dataset_names) != len(dataset_config_names):
        raise ValueError(f"Ensure one config is passed for each dataset, got {len(dataset_names)} datasets and {len(dataset_config_names)} configs.")

    splits = splits if splits is not None else ["train" for i in range(len(dataset_names))]
    text_column_names = text_column_names if text_column_names is not None else ["text" for i in range(len(dataset_names))]
    audio_column_names = audio_column_names if audio_column_names is not None else ["audio" for i in range(len(dataset_names))]

    dataset_splits = []
    # iterate over the datasets we want to interleave
    for i, dataset_name in enumerate(dataset_names):
        dataset = load_dataset(dataset_name, dataset_config_names[i], split=splits[i], streaming=True, **kwargs)
        # force audio column to sr=16000
        dataset = dataset.cast_column(audio_column_names[i], Audio(16000))
        # only keep the audio and text columns. Currently, remove_columns is BROKEN: https://github.com/huggingface/datasets/issues/5284, awaiting fix
        dataset = dataset.remove_columns(set(dataset.features.keys()) - set([audio_column_names[i], text_column_names[i]]))
        dataset_splits.append(dataset)

    interleaved_dataset = interleave_datasets(dataset_splits)
    return interleaved_dataset

In [4]:
dataset = load_multiple_streaming_datasets(dataset_names, configs, splits=splits, text_column_names=text_column_names, audio_column_names=audio_column_names, use_auth_token=True)

ValueError: The features can't be aligned because the key audio of features {'audio': {'array': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'path': Value(dtype='string', id=None), 'sampling_rate': Value(dtype='int64', id=None)}} has unexpected type - {'array': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'path': Value(dtype='string', id=None), 'sampling_rate': Value(dtype='int64', id=None)} (expected either {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'path': Value(dtype='string', id=None), 'sampling_rate': Value(dtype='int64', id=None)} or Value("null").